# NeuroTrain量化工具使用示例

本notebook展示了如何使用NeuroTrain框架的量化功能，包括多种量化方法和效果分析。


In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import logging

# 设置中文字体
plt.rcParams["font.sans-serif"] = ["DejaVu Sans"]
plt.rcParams["axes.unicode_minus"] = False

# 设置日志
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 导入量化工具
import sys

sys.path.append("..")
from src.quantization import (
    QuantizationConfig,
    QuantizationManager,
    QuantizationAnalyzer,
)

## 1. 创建示例模型和数据


In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


# 创建模型
model = SimpleCNN()
print(f"模型创建完成，总参数: {sum(p.numel() for p in model.parameters()):,}")

## 2. 动态量化示例


In [ ]:
# 创建动态量化配置
dynamic_config = QuantizationConfig(method="dynamic", dtype="qint8")

# 应用动态量化
manager = QuantizationManager(dynamic_config)
quantized_model = manager.quantize_model(model)

# 获取模型信息
size_info = manager.get_model_size_info(quantized_model)
print(f"动态量化完成!")
print(f"模型大小: {size_info['model_size_mb']:.2f}MB")
print(f"量化方法: {size_info['quantization_method']}")

## 3. 量化效果分析
